# 04 LSTM Eğitimi

In [1]:
import pandas as pd
import sys
sys.path.append("..")
from src.config import PROJECT_ROOT
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader, random_split


In [2]:
df = pd.read_parquet(PROJECT_ROOT.joinpath('data/interim/sample.parquet'))
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1

data_array = df.values


In [ ]:
dataset = TrafficDataset(data_array, X_STEP, Y_STEP, pca_components=36)
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])
train_loader = get_dataloaders(batch_size=10, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=10)


ValueError: n_components=1720 must be between 0 and min(n_samples, n_features)=5 with svd_solver='full'

In [ ]:
Below are the minimal edits you need to make in each notebook to wire-up the new PCA inversion feature.

04_train_lstm.ipynb
Cell where you build the dataset (around In[3]): Before
python
dataset = TrafficDataset(data_array, X_STEP, Y_STEP)
After
python
# choose your number of components
PCA_COMPONENTS = 36  
dataset = TrafficDataset(
    data_array,
    X_STEP,
    Y_STEP,
    pca_components=PCA_COMPONENTS
)
Cell where you instantiate the predictor (around In[?], just before .train()): Before
python
predictor = TrafficPredictor('lstm', sensors, X_STEP, Y_STEP)
After
python
predictor = TrafficPredictor(
    'lstm',
    sensors,
    X_STEP,
    Y_STEP,
    pca_model=dataset.pca
)
scgnn_train.ipynb
Cell where you build the dataset (around cell 3): Before
python
dataset = TrafficDataset(data_array, X_STEP, Y_STEP)
After
python
# choose your number of components
PCA_COMPONENTS = 10  
dataset = TrafficDataset(
    data_array,
    X_STEP,
    Y_STEP,
    pca_components=PCA_COMPONENTS
)
Cell where you instantiate the predictor (around cell 5): Before
python
predictor = TrafficPredictor(
    'scgnn',
    sensors,
    X_STEP,
    Y_STEP,
    edge_index=edge_index
)
After
python
predictor = TrafficPredictor(
    'scgnn',
    sensors,
    X_STEP,
    Y_STEP,
    edge_index=edge_index,
    pca_model=dataset.pca
)
Once you’ve made those changes, rerun both notebooks—your training‐loop metrics (and later inference) will now be computed in the original sensor-space. Let me know how it goes!

In [ ]:
sensors = data_array.shape[1]
print(sensors)

In [ ]:
predictor = TrafficPredictor('lstm', sensors, X_STEP, Y_STEP)
predictor.train(train_loader, val_loader, epochs=50, 
                save_path=PROJECT_ROOT.joinpath("models/lstm.pt"), 
                save_optimizer=True)
predictor.show()
